In [4]:
import speech_recognition as sr
import math
from tqdm import tqdm
import os
import pandas as pd
import librosa

In [5]:
def convert2text(audios, seconds=30, noise=False):
    r = sr.Recognizer()
    archivos = os.listdir(audios)
    datos = []

    for archivo in archivos:
        if archivo.endswith('.wav'):
            audio_path = os.path.join(audios, archivo)
            with sr.AudioFile(audio_path) as source:
                duration = source.DURATION
                splits = math.ceil(duration / seconds)
            
            transcripcion = ""  
            
            for i in tqdm(range(splits)):
                with source as src:
                    if noise:
                        r.adjust_for_ambient_noise(src)
                    audio = r.record(src, offset=i * seconds, duration=seconds)
                
                try:
                    texto = r.recognize_google(audio, language="es-MX")
                    transcripcion += texto + " "
                    speech_speed = get_speech_speed(audio_path)
                except:
                    pass
                
            datos.append([archivo, transcripcion, speech_speed])
            
    df = pd.DataFrame(datos, columns=["Archivo", "Transcripcion", "Velocidad"])
    df = df.loc[df['Transcripcion'].notna() & (df['Transcripcion'] != '')]
    df.to_csv("transcripciones.csv", index=False, encoding='latin-1')


    
  

In [6]:
def get_speech_speed(audio_path):
    # Se carga el audio
    audio, sr = librosa.load(audio_path)
            
    # Extraer segmentos del audio
    speech_segments = librosa.effects.split(audio, top_db=20)
            
    # Calcular la duración de cada segmento
    segment_durations = librosa.frames_to_time(
        [segment[1] - segment[0] for segment in speech_segments],
        sr=sr
         )
            
    # Calculate the average speed of speech
    average_speed = sum(segment_durations) / len(segment_durations)
            
    return average_speed
        

In [9]:
%time
carpeta_audio = "C:/Users/DSTHREE/Documents/GITHUB/Speech2Text/GRABACIONES"
convert2text(carpeta_audio, seconds=10)

CPU times: total: 0 ns
Wall time: 0 ns


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.47it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:09<00:00,  1.21s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:10<00:00,  1.89it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.03s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.33it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.03s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.71it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:23<00:00,  1.30it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:52<00:00,  2.18s/it]
